In [10]:
import tensorflow as tf
import os

In [11]:
deepfashion_dataset_path = "/Users/manishashivshette/Documents/ManishaStudies/CMPE295A/deep-fashion-dataset/"

In [12]:
target_attribute_model_folder = "/Users/manishashivshette/Documents/ManishaStudies/CMPE295B/AttributeModel/"

In [13]:
target_attribute_id = 1 # Refer to below deep_attr dictionary to train different attr models

In [62]:
num_epochs = 10
num_steps = 10

In [60]:
deep_attr={1:"texture",2:"fabric",3:"shape",4:"part",5:"style"}

In [61]:
saved_model_path = deepfashion_dataset_path+"models/model-"+deep_attr[target_attribute_id]+".h5"

In [14]:
# Load the attributes names into memory.
with tf.gfile.FastGFile(os.path.join(deepfashion_dataset_path, "Anno/list_attr_cloth.txt"), "r") as f:
    attribute_name_lines = f.readlines()

# Process attribute names.
num_attrs = int(attribute_name_lines.pop(0).strip())
header_attr = attribute_name_lines.pop(0).split()
assert len(attribute_name_lines) == num_attrs, "Wrong number of attributes."
attribute_names = list(map((lambda s: s.lower().split()[0:-1]), attribute_name_lines))

In [15]:
import collections
images_hashmap=collections.defaultdict()
# Load the attribute labels into memory.
with tf.gfile.FastGFile(os.path.join(deepfashion_dataset_path, "Anno/list_attr_img.txt"), "r") as f:
    attribute_lines = f.readlines()

# Check images are all loaded.
num_images = int(attribute_lines.pop(0).strip())
header_attr = attribute_lines.pop(0).split()
assert len(attribute_lines) == num_images, "Wrong number of images."

# Assign attribute labels to existing image metadata.
for element in attribute_lines:
    columns = element.split()
    filename = columns[0]
    attributes = []
    for x, v in enumerate(columns[1:]):
        if int(v) == 1:
            attributes.extend(attribute_names[int(x)])
    attribute_ids = [a.strip().lower() for a in attributes]
    
    attribute_ids = [x for x in attribute_ids]
    
    images_hashmap[filename] = attribute_ids
print(images_hashmap)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
import shutil
import os
import re
import cv2
import splitter
import pandas as pd
import collections

In [17]:
import numpy as np
from six.moves import range

In [18]:
splitter = re.compile("\s+")
base_path = deepfashion_dataset_path + 'img'

In [20]:
attr_file = target_attribute_model_folder+ "list_attr_clothes.csv"

In [21]:
df = pd.read_csv(attr_file)
df.head()    

,attribute_name,attribute_type
0,a-line,3
1,abstract,1
2,abstract chevron,1
3,abstract chevron print,1
4,abstract diamond,1


In [22]:
#extract expected attributes
df_filted_attribute = df[df['attribute_type']==target_attribute_id]

In [23]:
df_filted_attribute.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215 entries, 22 to 999
Data columns (total 2 columns):
attribute_name      215 non-null object
attribute_type      215 non-null int64
dtypes: int64(1), object(1)
memory usage: 5.0+ KB


In [25]:
df_filted_attribute['attribute_name  ']=df_filted_attribute['attribute_name  '].str.strip()

/Users/manishashivshette/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
attribute_dict = dict(zip(df_filted_attribute['attribute_name  '], df_filted_attribute.attribute_type))

In [33]:
def process_folders():
    # Read the relevant annotation file and preprocess it
    # Assumed that the annotation files are under '<project folder>/data/anno' path
    partitition_file_path = deepfashion_dataset_path+"/Eval/list_eval_partition.txt"
    with open(partitition_file_path, 'r') as eval_partition_file:
        list_eval_partition = [line.rstrip('\n') for line in eval_partition_file][2:]
        list_eval_partition = [splitter.split(line) for line in list_eval_partition]
        list_all =[]
        for v in list_eval_partition:
            img_key = v[0][4:]
            img_key= "img/"+img_key
            if img_key in images_hashmap:
                for each_attr in images_hashmap[img_key]:
                    if each_attr in attribute_dict:
                        list_all.append((v[0][4:],each_attr , v[1]))
    print(list_all[0:5])
    count =0
    for element in list_all:
        if not os.path.exists(os.path.join(base_path, element[2])):
            os.mkdir(os.path.join(base_path, element[2]))
        if not os.path.exists(os.path.join(os.path.join(base_path, element[2]), element[1])):
            os.mkdir(os.path.join(os.path.join(base_path, element[2]), element[1]))
        if not os.path.exists(os.path.join(os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1])),
                              element[0].split('/')[0])):
            print(os.path.join(os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1])),
                     element[0].split('/')[0]))
            os.mkdir(os.path.join(os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1])),
                     element[0].split('/')[0]))
        try:
            
            shutil.copyfile(os.path.join(base_path, element[0]),
                os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1]), element[0]))
        except:
            count+=1
            #print(os.path.join(base_path, element[0]))
            #print("one file not found")
    print("file not found count:",count) 

In [34]:
process_folders()

[('Sheer_Pleated-Front_Blouse/img_00000003.jpg', 'sleeveless', 'val'), ('Sheer_Pleated-Front_Blouse/img_00000003.jpg', 'v-neck', 'val'), ('Sheer_Pleated-Front_Blouse/img_00000008.jpg', 'hem', 'train'), ('Sheer_Pleated-Front_Blouse/img_00000015.jpg', 'contrast', 'val'), ('Sheer_Pleated-Front_Blouse/img_00000015.jpg', 'lapel', 'val')]
file not found count: 9085


In [35]:
# move train, test and val folders to target attr model folder

shutil.move(base_path+"/train", target_attribute_model_folder+"data/") 

'/Users/manishashivshette/Documents/ManishaStudies/CMPE295B/AttributeModel//data/train'

In [36]:
shutil.move(base_path+"/test", target_attribute_model_folder+"data/") 
shutil.move(base_path+"/val", target_attribute_model_folder+"data/") 

'/Users/manishashivshette/Documents/ManishaStudies/CMPE295B/AttributeModel/data/val'

In [37]:
# get the bounding boxes for each image, preprocessing function is in other file, use these files directly now-- Manisha
train_df = pd.read_csv("dict_train.csv")
dict_train = train_df.to_dict()
test_df = pd.read_csv("dict_test.csv")
dict_test = test_df.to_dict()
val_df = pd.read_csv("dict_val.csv")
dict_val = val_df.to_dict()

In [38]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import DirectoryIterator, ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras import backend as K

In [39]:
model_resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg')

In [40]:
for layer in model_resnet.layers[:-12]:
    # 6 - 12 - 18 have been tried. 12 is the best.
    layer.trainable = False

In [48]:
num_classes = len(os.listdir(target_attribute_model_folder+"data/train"))
print("Attribute classes:",num_classes)

Attribute classes: 135


In [50]:
x = model_resnet.output
x = Dense(512, activation='elu', kernel_regularizer=l2(0.001))(x)
# last layer to classify the image into one of the attribute
y = Dense(num_classes, activation='softmax', name='img')(x)

In [51]:
x_bbox = model_resnet.output
x_bbox = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x_bbox)
x_bbox = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x_bbox)
bbox = Dense(4, kernel_initializer='normal', name='bbox')(x_bbox)

In [52]:
final_model = Model(inputs=model_resnet.input,
                    outputs=[y, bbox])

In [53]:
print(final_model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [54]:
class DirectoryIteratorWithBoundingBoxes(DirectoryIterator):
    def __init__(self, directory, image_data_generator, bounding_boxes: dict = None, target_size=(256, 256),
                 color_mode: str = 'rgb', classes=None, class_mode: str = 'categorical', batch_size: int = 32,
                 shuffle: bool = True, seed=None, data_format=None, save_to_dir=None,
                 save_prefix: str = '', save_format: str = 'jpeg', follow_links: bool = False):
        super().__init__(directory, image_data_generator, target_size, color_mode, classes, class_mode, batch_size,
                         shuffle, seed, data_format, save_to_dir, save_prefix, save_format, follow_links)
        self.bounding_boxes = bounding_boxes

    def next(self):
        """
        # Returns
            The next batch.
        """
        with self.lock:
            index_array = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype=K.floatx())
        locations = np.zeros((len(batch_x),) + (4,), dtype=K.floatx())

        grayscale = self.color_mode == 'grayscale'
        # build batch of image data
        for i, j in enumerate(index_array):
            fname = self.filenames[j]
            img = image.load_img(os.path.join(self.directory, fname),
                                 grayscale=grayscale,
                                 target_size=self.target_size)
            x = image.img_to_array(img, data_format=self.data_format)
            x = self.image_data_generator.random_transform(x)
            x = self.image_data_generator.standardize(x)
            batch_x[i] = x

            if self.bounding_boxes is not None and fname in self.bounding_boxes :
                bounding_box = self.bounding_boxes[fname]
                locations[i] = np.asarray(
                    [bounding_box['x1'], bounding_box['y1'], bounding_box['x2'], bounding_box['y2']],
                    dtype=K.floatx())
        # optionally save augmented images to disk for debugging purposes
        # build batch of labels
        if self.class_mode == 'sparse':
            batch_y = self.classes[index_array]
        elif self.class_mode == 'binary':
            batch_y = self.classes[index_array].astype(K.floatx())
        elif self.class_mode == 'categorical':
            batch_y = np.zeros((len(batch_x), num_classes), dtype=K.floatx())
            for i, label in enumerate(self.classes[index_array]):
                batch_y[i, label] = 1.
        else:
            return batch_x

        if self.bounding_boxes is not None:
            return batch_x, [batch_y, locations]
        else:
            return batch_x, batch_y

In [55]:
opt = SGD(lr=0.0001, momentum=0.9, nesterov=True)

In [56]:
final_model.compile(optimizer=opt,
                    loss={'img': 'categorical_crossentropy',
                          'bbox': 'mean_squared_error'},
                    metrics={'img': ['accuracy', 'top_k_categorical_accuracy'], # default: top-5
                             'bbox': ['mse']})

In [57]:
train_datagen = ImageDataGenerator(rotation_range=30.,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator()

In [66]:
train_data_path = target_attribute_model_folder + "data/train/"
train_iterator = DirectoryIteratorWithBoundingBoxes(train_data_path, train_datagen, bounding_boxes=dict_train, target_size=(200, 200))
val_data_path = target_attribute_model_folder + "data/val/"
val_iterator = DirectoryIteratorWithBoundingBoxes(val_data_path, test_datagen, bounding_boxes=dict_val,target_size=(200, 200))

Found 105474 images belonging to 135 classes.
Found 17840 images belonging to 135 classes.


In [67]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               patience=12,
                               factor=0.5,
                               verbose=1)
tensorboard = TensorBoard(log_dir='./logs')
early_stopper = EarlyStopping(monitor='val_loss',
                              patience=30,
                              verbose=1)


checkpoint = ModelCheckpoint(saved_model_path)

In [68]:
def custom_generator(iterator):
    while True:
        batch_x, batch_y = iterator.next()
        yield (batch_x, batch_y)

In [ ]:
history = final_model.fit_generator(custom_generator(train_iterator),
                          steps_per_epoch=num_steps,
                          epochs=num_epochs, validation_data=custom_generator(val_iterator),
                          validation_steps=1000,
                          verbose=1,
                          shuffle=True,
                          callbacks=[lr_reducer, checkpoint, early_stopper, tensorboard],
                          workers=12)

Epoch 1/10
 9/10 [==========================>...] - ETA: 4s - loss: 7.0654 - img_loss: 5.1843 - bbox_loss: 0.0377 - img_acc: 0.0035 - img_top_k_categorical_accuracy: 0.0278 - bbox_mean_squared_error: 0.0377

In [ ]:
import matplotlib.pyplot as plt
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [65]:
test_datagen = ImageDataGenerator()
test_data_path = target_attribute_model_folder + "data/test/"
test_iterator = DirectoryIteratorWithBoundingBoxes(test_data_path, test_datagen, bounding_boxes=dict_test, target_size=(200, 200))
scores = final_model.evaluate_generator(custom_generator(test_iterator), steps=10)

print('Multi target loss: ' + str(scores[0]))
print('Image loss: ' + str(scores[1]))
print('Bounding boxes loss: ' + str(scores[2]))
print('Image accuracy: ' + str(scores[3]))
print('Top-5 image accuracy: ' + str(scores[4]))
print('Bounding boxes error: ' + str(scores[5]))

Found 20209 images belonging to 87 classes.
Multi target loss: 6.2155430793762205
Image loss: 4.3292248249053955
Bounding boxes loss: 0.04588926397264004
Image accuracy: 0.071875
Top-5 image accuracy: 0.209375
Bounding boxes error: 0.04588926397264004
